<b> This notebook is needed for creating the data set needed to train a severity classifier. It takes input the main data set (motor vehicles collison in NYC) and import the following weather based columns into it:</b>
<br><br>
<b>Temperature_2m </b>: Temperative 2 meters above ground. <br>
<b>Relative_humidity_2m</b> : Relative Humidity 2 meters above ground. <br> 
<b>Precipitation</b> : Total precipitation (rain, showers, snow) sum of the preceding hour. <br>
<b>Rain</b>: Only liquid precipitation of the preceding hour including local showers and rain from large scale systems. <br> 
<b>Snowfall</b>:  Snowfall amount of the preceding hour in centimeters. <br> 
<b>Snow_depth</b>: Snow depth on the ground. <br> 
<b>Weather_code</b>: WMO standard weather code. <br>
<b>Cloud_cover</b>: Total cloud cover as an area fraction.<br>
<b>Wind_speed_100m</b>: Wind speed at 100 meters above ground.<br>
<b>Wind_direction_100m</b>: Wind direction at 100 meters above ground.<br>
<b>Is_day</b>: Boolean variable for whether its day or night. <br>

Visit https://open-meteo.com/en/docs/historical-weather-api for more details.

# Installing Dependencies


In [1]:
# Install dependencies
!pip install openmeteo_requests
!pip install requests-cache retry-requests

In [2]:
# Import all the required packages
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import numpy as np
import time

In [2]:
# The dataset is located at https://catalog.data.gov/dataset/motor-vehicle-collisions-crashes
# Mount google drive where the input csv is located
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Read the input csv into a data frame
df_base = pd.read_csv('/content/drive/MyDrive/FDS_Project/final_imputation.csv')

In [4]:
df_base.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,STRTNAME_BOROUGH
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,"WHITESTONE EXPRESSWAY NYC, New York",20 AVENUE,NaN,...,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,"QUEENSBORO BRIDGE UPPER NYC, New York",NaN,NaN,...,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,"THROGS NECK BRIDGE NYC, New York",NaN,NaN,...,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)","SARATOGA AVENUE NYC, New York",DECATUR STREET,NaN,...,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,"SARATOGA AVENUE NYC, New York, BROOKLYN"


# Weather data import
The rest of the code deals with importing the weather data correctly and joining those columns with the main data set: <br>
The blocks below can be logically broken down into these functions: <br>
1) Determine the min date and max date for which openmeteo API needs to be executed<br>
2) Determine the 49 representative lat longs for which openmeteo API needs to be executed<br>
3) Run the openmeteo API in 2 batches over 2 days to prevent rate limiting.<br>
4) Replace the original lat longs with representative lat longs based on closest distance<br>
5) Do some cleanup, like removing rows with missing lat longs and standardize date and time format<br>
6) Join the weather data columns with the main data set.


In [ ]:
# Fix the format of CRASH DATE TO YYYY-MM-DD
# and of the CRASH TIME to HH::MM and round it to the nearest hour

df_base['CRASH DATE'] = pd.to_datetime(df_base['CRASH DATE'])
df_base['CRASH TIME'] = pd.to_datetime(df_base['CRASH TIME'])

# Round the values in the 'CRASH TIME' column to the nearest hour
df_base['CRASH TIME'] = df_base['CRASH TIME'].dt.round('H')


In [11]:
# Print the max date and min date in the data set
# This is used to determine 
min_date = df_base['CRASH DATE'].min()
max_date = df_base['CRASH DATE'].max()

print(f"Minimum Date: {min_date}")
print(f"Maximum Date: {max_date}")

Minimum Date: 2012-07-01 00:00:00
Maximum Date: 2023-10-14 00:00:00


In [12]:
# Extract year from the date and count unique entries for each year
unique_entries_per_year = df_base.groupby(df_base['CRASH DATE'].dt.year)['CRASH DATE'].nunique()

# Display the result
print(unique_entries_per_year)

CRASH DATE
2012    184
2013    365
2014    365
2015    365
2016    366
2017    365
2018    365
2019    365
2020    366
2021    365
2022    365
2023    287
Name: CRASH DATE, dtype: int64


In [23]:
# This is a function written to call the openmeteo API and get the weather data
# @input - location latitude, location longitude, start date and end date
# @output - Gives the 24 hour hourly weather data (temp, humidity etc., as below) for all dates
# between the start date and end date for that latitude and longitude
def get_historical_weather(latitude, longitude, start_date, end_date):
  # Setup the Open-Meteo API client with cache and retry on error
  cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
  retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
  openmeteo = openmeteo_requests.Client(session = retry_session)

  # Make sure all required weather variables are listed here
  # The order of variables in hourly or daily is important to assign them correctly below
  url = "https://archive-api.open-meteo.com/v1/archive"
  params = {
	  "latitude": latitude,
	  "longitude": longitude,
	  "start_date": start_date,
	  "end_date": end_date,
	  "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "cloud_cover", "wind_speed_100m", "wind_direction_100m", "is_day"]
  }
  responses = openmeteo.weather_api(url, params=params)

  response = responses[0]

  # Process hourly data. The order of variables needs to be the same as requested.
  hourly = response.Hourly()
  hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
  hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
  hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
  hourly_rain = hourly.Variables(3).ValuesAsNumpy()
  hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
  hourly_snow_depth = hourly.Variables(5).ValuesAsNumpy()
  hourly_weather_code = hourly.Variables(6).ValuesAsNumpy()
  hourly_cloud_cover = hourly.Variables(7).ValuesAsNumpy()
  hourly_wind_speed_100m = hourly.Variables(8).ValuesAsNumpy()
  hourly_wind_direction_100m = hourly.Variables(9).ValuesAsNumpy()
  hourly_is_day = hourly.Variables(10).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
	  start = pd.to_datetime(hourly.Time(), unit = "s"),
	  end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
    freq = pd.Timedelta(seconds = hourly.Interval()),
	  inclusive = "left"
  )}

  hourly_data["temperature_2m"] = hourly_temperature_2m
  hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
  hourly_data["precipitation"] = hourly_precipitation
  hourly_data["rain"] = hourly_rain
  hourly_data["snowfall"] = hourly_snowfall
  hourly_data["snow_depth"] = hourly_snow_depth
  hourly_data["weather_code"] = hourly_weather_code
  hourly_data["cloud_cover"] = hourly_cloud_cover
  hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
  hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
  hourly_data["is_day"] = hourly_is_day

  hourly_dataframe = pd.DataFrame(data = hourly_data)
  return hourly_dataframe

In [6]:
# As the number of unique lat longs in the data set is huge (>1.8 million), getting 24 hour
# weather data for all from open meteo is very expensive and unnecesary. We use the fact 
# that the weather conditions between two very close lat longs will not change significantly
# and we can use representative lat longs to replace the actual lat longs. For this,
# we create a 7x7 grid across NYC = 49 representative lat longs. We get all the weather
# data for these 49 lat longs from 2012 - 2023 using openmeteo, and fill in that information
# for the actual lat long based on closest distance to the representative lat long. We use
# cdist from scipy.spatial.distance package to do this.


# Define the bounding box for NYC
lat_min, lat_max = 40.4774, 40.9176
lon_min, lon_max = -74.2591, -73.7004

# Number of points in each dimension
num_points = 7

# Generate evenly spaced coordinates
latitudes = np.linspace(lat_min, lat_max, num_points)
longitudes = np.linspace(lon_min, lon_max, num_points)

# Create a meshgrid and flatten the coordinates
lat_mesh, lon_mesh = np.meshgrid(latitudes, longitudes)
flat_latitudes = lat_mesh.flatten()
flat_longitudes = lon_mesh.flatten()

# Display the selected coordinates
coordinates = list(zip(flat_latitudes, flat_longitudes))
for coord in coordinates:
    print(coord)


(40.4774, -74.2591)
(40.55076666666667, -74.2591)
(40.62413333333333, -74.2591)
(40.697500000000005, -74.2591)
(40.77086666666667, -74.2591)
(40.844233333333335, -74.2591)
(40.9176, -74.2591)
(40.4774, -74.16598333333334)
(40.55076666666667, -74.16598333333334)
(40.62413333333333, -74.16598333333334)
(40.697500000000005, -74.16598333333334)
(40.77086666666667, -74.16598333333334)
(40.844233333333335, -74.16598333333334)
(40.9176, -74.16598333333334)
(40.4774, -74.07286666666667)
(40.55076666666667, -74.07286666666667)
(40.62413333333333, -74.07286666666667)
(40.697500000000005, -74.07286666666667)
(40.77086666666667, -74.07286666666667)
(40.844233333333335, -74.07286666666667)
(40.9176, -74.07286666666667)
(40.4774, -73.97975)
(40.55076666666667, -73.97975)
(40.62413333333333, -73.97975)
(40.697500000000005, -73.97975)
(40.77086666666667, -73.97975)
(40.844233333333335, -73.97975)
(40.9176, -73.97975)
(40.4774, -73.88663333333334)
(40.55076666666667, -73.88663333333334)
(40.62413333333

In [24]:
# Get all the weather data for the 49 representative lat longs using
# the get_historical_weather() function. The openmeteo API has a rate
# limit on no of API calls / minute, /hour and /day. So we used thread
# sleeps but still ran out of calls/day. This block was thus run in
# batches over a period of two days. 

min_date = "2012-07-01" # Determined earlier in EDA
max_date = "2023-10-14" # Determined earlier in EDA
base_df = pd.DataFrame()
try:
  for idx, coord in enumerate(coordinates):
    print("Running for coord no: ", idx)
    latitude = coord[0]
    longitude = coord[1]
    print(latitude, longitude)
    new_df = get_historical_weather(latitude,longitude,min_date,max_date)
    base_df = pd.concat([base_df, new_df], ignore_index=True)
    # Rate limiting  
    time.sleep(90)
except Exception as e:
  print(f"An error occurred: {e}")

Running for coord no:  16
40.62413333333333 -74.07286666666667
Running for coord no:  17
40.697500000000005 -74.07286666666667
Running for coord no:  18
40.77086666666667 -74.07286666666667
Running for coord no:  19
40.844233333333335 -74.07286666666667
Running for coord no:  20
40.9176 -74.07286666666667
Running for coord no:  21
40.4774 -73.97975
Running for coord no:  22
40.55076666666667 -73.97975
Running for coord no:  23
40.62413333333333 -73.97975
Running for coord no:  24
40.697500000000005 -73.97975
Running for coord no:  25
40.77086666666667 -73.97975
Running for coord no:  26
40.844233333333335 -73.97975
Running for coord no:  27
40.9176 -73.97975
Running for coord no:  28
40.4774 -73.88663333333334
Running for coord no:  29
40.55076666666667 -73.88663333333334
Running for coord no:  30
40.62413333333333 -73.88663333333334
Running for coord no:  31
40.697500000000005 -73.88663333333334
An error occurred: {'error': True, 'reason': 'Daily API request limit exceeded. Please try

In [ ]:
# This is the code to fix the weather data returned by openmeteo
# This code appends the lat long with the weather data returned by openmeteo
# This code works on the two batch files returned by the previous
# block. It begins by horizontally concatenating them and then
# adding representative lat longs as two columns to that data frame

# Read the two CSV files into separate DataFrames
# These are the two CSVs created from the two DataFrames generated by the previous block
df1 = pd.read_csv('/content/drive/MyDrive/FDS_Project/weather-data-1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/FDS_Project/weather-data-2.csv')

# Concatenate the two DataFrames into a single DataFrame
dfm = pd.concat([df1, df2], ignore_index=True)


base_data = {
    'latitude': [],
    'longitude': []
}
lat_long_df = pd.DataFrame(base_data)
# Append rows to the base DataFrame in chunks of 98,952
# This is the number of weather data rows for each unique
# representative lat long. For each day from min_date = "2012-07-01" 
# to max_date = "2023-10-14"
chunk_size = 98952

for idx, coord in enumerate(coordinates):
    # Create static latitude and longitude series
    static_latitudes = pd.Series(np.full(chunk_size, coord[0]), name='latitude')
    static_longitudes = pd.Series(np.full(chunk_size, coord[1]), name='longitude')

    # Create a chunk DataFrame
    chunk_df = pd.DataFrame()

    # Concatenate the static latitude and longitude series with the chunk DataFrame
    chunk_df = pd.concat([static_latitudes, static_longitudes], axis=1)

    # Append the chunk DataFrame to the base DataFrame
    lat_long_df = pd.concat([lat_long_df, chunk_df], ignore_index=True)

# Append columns of the base DataFrame to another DataFrame
dfm = pd.concat([dfm, lat_long_df[['latitude', 'longitude']]], axis=1)

# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(dfm.head())




In [13]:
# Display the updated DataFrame
dfm.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,snow_depth,weather_code,cloud_cover,wind_speed_100m,wind_direction_100m,is_day,latitude,longitude
0,2012-07-01 00:00:00,29.8975,32.264313,0.0,0.0,0.0,0.0,1.0,47.100002,18.723460,271.10168,1.0,40.4774,-74.2591
1,2012-07-01 01:00:00,28.3475,35.523857,0.0,0.0,0.0,0.0,0.0,11.400001,21.288757,273.87845,0.0,40.4774,-74.2591
2,2012-07-01 02:00:00,26.8475,40.212280,0.0,0.0,0.0,0.0,0.0,13.500001,22.253124,279.30988,0.0,40.4774,-74.2591
3,2012-07-01 03:00:00,25.6475,44.314785,0.0,0.0,0.0,0.0,0.0,7.500000,21.434364,277.72174,0.0,40.4774,-74.2591
4,2012-07-01 04:00:00,24.5475,48.889630,0.0,0.0,0.0,0.0,0.0,0.300000,21.647945,273.81400,0.0,40.4774,-74.2591


In [17]:
# Capitalize the column names of the weather df (dfm) to match the standard in the 
# original accident data (df_base)
dfm.columns = dfm.columns.str.upper()
dfm.head()

,DATE,TEMPERATURE_2M,RELATIVE_HUMIDITY_2M,PRECIPITATION,RAIN,SNOWFALL,SNOW_DEPTH,WEATHER_CODE,CLOUD_COVER,WIND_SPEED_100M,WIND_DIRECTION_100M,IS_DAY,LATITUDE,LONGITUDE,CRASH DATE,CRASH TIME
0,2012-07-01 00:00:00,29.8975,32.264313,0.0,0.0,0.0,0.0,1.0,47.100002,18.723460,271.10168,1.0,40.4774,-74.2591,2012-07-01,00:00:00
1,2012-07-01 01:00:00,28.3475,35.523857,0.0,0.0,0.0,0.0,0.0,11.400001,21.288757,273.87845,0.0,40.4774,-74.2591,2012-07-01,01:00:00
2,2012-07-01 02:00:00,26.8475,40.212280,0.0,0.0,0.0,0.0,0.0,13.500001,22.253124,279.30988,0.0,40.4774,-74.2591,2012-07-01,02:00:00
3,2012-07-01 03:00:00,25.6475,44.314785,0.0,0.0,0.0,0.0,0.0,7.500000,21.434364,277.72174,0.0,40.4774,-74.2591,2012-07-01,03:00:00
4,2012-07-01 04:00:00,24.5475,48.889630,0.0,0.0,0.0,0.0,0.0,0.300000,21.647945,273.81400,0.0,40.4774,-74.2591,2012-07-01,04:00:00


In [10]:
# Code to replace the original lat longs in the data set with
# the 7x7 representative lat longs. This is based on closest distance
# This is needed to JOIN the main data set with the weather columns data set
# NOTE: The data set still contains the original lat longs in the
# 'LOCATION' column if needed for training.

from scipy.spatial.distance import cdist
# clearer naming
main_df = df_base

# Drop all rows with missing lat long
main_df = main_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# representative coordinates
representative_coords = coordinates

# Function to find the index of the closest representative coordinate
def find_closest_coordinate(row):
    distances = cdist([[row['LATITUDE'], row['LONGITUDE']]], representative_coords)
    closest_index = distances.argmin()
    return closest_index

# Apply the function to each row to find the closest representative coordinate index
main_df['closest_index'] = main_df.apply(find_closest_coordinate, axis=1)

main_df[['LATITUDE', 'LONGITUDE']] = main_df['closest_index'].apply(lambda idx: pd.Series(representative_coords[idx]))

# Drop the intermediate 'closest_index' column
main_df = main_df.drop('closest_index', axis=1)

# Display the updated main DataFrame
print("Updated Main DataFrame:")
print(main_df.head())


<ipython-input-10-59ca1cf9f6d5>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['closest_index'] = main_df.apply(find_closest_coordinate, axis=1)
<ipython-input-10-59ca1cf9f6d5>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df[['LATITUDE', 'LONGITUDE']] = main_df['closest_index'].apply(lambda idx: pd.Series(representative_coords[idx]))


Updated Main DataFrame:
  CRASH DATE          CRASH TIME   BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
3 2021-09-11 2023-11-30 10:00:00  BROOKLYN  11208.0  40.697500 -73.886633   
4 2021-12-14 2023-11-30 08:00:00  BROOKLYN  11233.0  40.697500 -73.886633   
6 2021-12-14 2023-11-30 17:00:00       NaN      NaN  40.697500 -73.979750   
7 2021-12-14 2023-11-30 08:00:00     BRONX  10475.0  40.844233 -73.793517   
8 2021-12-14 2023-11-30 21:00:00  BROOKLYN  11207.0  40.697500 -73.886633   

                  LOCATION                             ON STREET NAME  \
3    (40.667202, -73.8665)                                        NaN   
4  (40.683304, -73.917274)             SARATOGA AVENUE NYC,  New York   
6  (40.709183, -73.956825)  BROOKLYN QUEENS EXPRESSWAY NYC,  New York   
7    (40.86816, -73.83148)                                        NaN   
8     (40.67172, -73.8971)                                        NaN   

  CROSS STREET NAME              OFF STREET NAME  ...  \
3               N

In [15]:
# Reformat crash date and time to the format in the weather df
# This is needed for JOIN
dfm['CRASH DATE'] = dfm['date'].dt.strftime('%Y-%m-%d')
dfm['CRASH TIME'] = dfm['date'].dt.strftime('%H:%M:%S')
main_df.head()


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,STRTNAME_BOROUGH
3,2021-09-11,10:00:00,BROOKLYN,11208.0,40.697500,-73.886633,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,NaN
4,2021-12-14,08:00:00,BROOKLYN,11233.0,40.697500,-73.886633,"(40.683304, -73.917274)","SARATOGA AVENUE NYC, New York",DECATUR STREET,NaN,...,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,"SARATOGA AVENUE NYC, New York, BROOKLYN"
6,2021-12-14,17:00:00,NaN,NaN,40.697500,-73.979750,"(40.709183, -73.956825)","BROOKLYN QUEENS EXPRESSWAY NYC, New York",NaN,NaN,...,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,NaN
7,2021-12-14,08:00:00,BRONX,10475.0,40.844233,-73.793517,"(40.86816, -73.83148)",NaN,NaN,344 BAYCHESTER AVENUE,...,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN,NaN
8,2021-12-14,21:00:00,BROOKLYN,11207.0,40.697500,-73.886633,"(40.67172, -73.8971)",NaN,NaN,2047 PITKIN AVENUE,...,NaN,NaN,NaN,4487074,Sedan,NaN,NaN,NaN,NaN,NaN


In [19]:
# Merge the original data set with the weather data using INNER Join on the PKey columns
result_df = pd.merge(main_df, dfm, on=['LATITUDE', 'LONGITUDE', 'CRASH DATE', 'CRASH TIME'], how='inner')

In [21]:
# Convert final JOINED df to a csv
result_df.to_csv('final_data.csv', index=False)

In [22]:
# The csv is huge, so zip it and store in google drive
!zip -r /content/final_data.zip /content/final_data.csv

  adding: content/final_data.csv (deflated 85%)
